# MIMIC III Inhospital Mortality Benchmark Cohort

### Import packages

In [ ]:
#import required packages
import json 
import pandas as pd
import numpy as np
from rpy2.robjects import r
from rpy2 import robjects as ro
from rpy2.robjects.packages import importr

from rpy2.robjects import pandas2ri
pandas2ri.activate()

base = importr('base')
database_connector = importr('DatabaseConnector')

### Specify details for connecting to the server and feating data

In [ ]:
# Specify details for connecting to the server
dbms = ''
path_to_driver = ''
user = ''
pw = ''
server = ''
port = ''

create_connection_details = database_connector.createConnectionDetails

connection_details = create_connection_details(dbms = dbms,
                                               pathToDriver = path_to_driver,
                                               server = server,
                                               user = user,
                                               password = pw,
                                               port = port)

In [ ]:
def fetch_data(query):
    connect = database_connector.connect
    query_sql = database_connector.querySql
    conn = connect(connection_details)
    return query_sql(conn, query)

#### Define the target cohort

In [ ]:
def get_patients():
    query = """
               SELECT *
               FROM omop2.person
            
            """
    return fetch_data(query)
patients = get_patients()
patients

In [ ]:
fetch_data("select * from pg_indexes where schemaname ='omop2'")

In [ ]:
def get_hosp_admissions():
    query = """ 
                SELECT * 
                FROM omop2.visit_occurrence 
                WHERE visit_concept_id IN (9201, 9203, 262)
                
            """
    return fetch_data(query)
hosp_admissions = get_hosp_admissions()
hosp_admissions 

In [ ]:
def get_counts(icustays, counts, step = 'root', prev_step = None):
    
    if step == 'root':
        counts[step] = dict()
        counts[step]['included'] = dict()
        counts[step]['included']['patients'] = len(icustays['PERSON_ID'].unique())
        counts[step]['included']['hospitalizations'] = len(icustays['VISIT_OCCURRENCE_ID'].unique())
        counts[step]['included']['icustays'] = len(icustays['VISIT_DETAIL_ID'].unique())
    else:
        counts[step] = dict()
        counts[step]['included'] = dict() 
        counts[step]['included'] ['patients'] = len(icustays['PERSON_ID'].unique())
        counts[step]['included'] ['hospitalizations'] = len(icustays['VISIT_OCCURRENCE_ID'].unique())
        counts[step]['included'] ['icustays'] = len(icustays['VISIT_DETAIL_ID'].unique())

        counts[step]['excluded'] = dict() 
        counts[step]['excluded'] ['patients'] = counts[prev_step]['included'] ['patients'] - counts[step]['included'] ['patients']
        counts[step]['excluded'] ['hospitalizations'] = counts[prev_step]['included'] ['hospitalizations'] - counts[step]['included'] ['hospitalizations']
        counts[step]['excluded'] ['icustays'] = counts[prev_step]['included'] ['icustays'] - counts[step]['included'] ['icustays']

    return counts
    


In [ ]:

def get_icustays():
    """Get all ICU stays"""
    
    query = """
                SELECT *
                From omop2.visit_detail 
                WHERE visit_detail_concept_id = 32037
                AND visit_type_concept_id = 2000000006
            
            """
    
    icustays = fetch_data(query)
    
    return icustays

icustays = get_icustays()
icustays

In [ ]:
counts = dict()
counts = get_counts(icustays, counts, step = 'root')
counts

In [ ]:
def exclude_multiple_icustays(icustays):
    """Exclude 2+ ICU stays per admission and multiple ICU transfers"""
    icustays_per_admission = icustays['VISIT_OCCURRENCE_ID'].value_counts().reset_index().rename(columns ={'index':'VISIT_OCCURRENCE_ID','VISIT_OCCURRENCE_ID':'count'})
    to_keep = icustays_per_admission['VISIT_OCCURRENCE_ID'][icustays_per_admission['count']==1]
    
    icustays = icustays[icustays['VISIT_OCCURRENCE_ID'].isin(to_keep)].copy()
    return icustays

filtered_icustays = exclude_multiple_icustays(icustays)
filtered_icustays 

In [ ]:
counts = get_counts(filtered_icustays , counts, 'step1_exclude_multiple_stays','root')
counts 

In [ ]:
def add_age_to_icustays(patients, icustays):
    icustays['BIRTH_DATETIME'] = [patients['BIRTH_DATETIME'][patients['PERSON_ID']== i].iloc[0] for i in icustays['PERSON_ID']]
    age_in_days = icustays['VISIT_START_DATETIME'].dt.to_pydatetime() - icustays['BIRTH_DATETIME'].dt.to_pydatetime()
    icustays['AGE'] = [i.days/365.25 for i in age_in_days]
    icustays.loc[icustays['AGE'] < 0, 'AGE'] = 90
    return icustays
filtered_icustays = add_age_to_icustays(patients, filtered_icustays)
filtered_icustays                                        

In [ ]:
def exclude_pediatrics(icustays, min_age=18, max_age=np.inf):
    """Exclude Pediatric patients"""
    icustays = icustays[(icustays['AGE'] >= min_age) & (icustays['AGE'] <= max_age)]
    return icustays

filtered_icustays = exclude_pediatrics(filtered_icustays)
filtered_icustays

In [ ]:
counts = get_counts(filtered_icustays, counts, 'step2_exclude_pediatric_patients','step1_exclude_multiple_stays')
counts

In [ ]:
def exclude_icustays_on_length_of_stay(icustays, min_los=48, max_los=np.inf):
    """Exclude ICU stays with length-of-stay less than 48 hours"""
    icustays= icustays.copy()
    icustays['LENGTH_OF_STAY'] = icustays['VISIT_END_DATETIME'].dt.to_pydatetime() - icustays['VISIT_START_DATETIME'].dt.to_pydatetime()
    icustays['LENGTH_OF_STAY'] =icustays['LENGTH_OF_STAY']/np.timedelta64(1, 'h')
    icustays = icustays[(icustays['LENGTH_OF_STAY'] >= min_los) & (icustays['LENGTH_OF_STAY'] <= max_los)]
    return icustays

filtered_icustays = exclude_icustays_on_length_of_stay(filtered_icustays)
filtered_icustays

In [ ]:
counts = get_counts(filtered_icustays, counts, 'step3_exclude_icustays_less_than_48_hours','step2_exclude_pediatric_patients')
counts 

In [ ]:
def exclude_icustays_with_no_events(icustays):
    """Exclude ICU stays with no events before 48 hours"""
    query = """
               SELECT distinct A.person_id, A.visit_detail_id
               FROM omop2.measurement A
               inner join omop2.visit_detail B on A.VISIT_DETAIL_ID = B.VISIT_DETAIL_ID
               AND A.measurement_datetime <= B.visit_start_datetime + INTERVAL '2 day'

            """
    to_keep = fetch_data(query)
    return icustays[icustays['VISIT_DETAIL_ID'].isin(to_keep['VISIT_DETAIL_ID'])].copy()
filtered_icustays = exclude_icustays_with_no_events(filtered_icustays)
filtered_icustays

In [ ]:
counts = get_counts(filtered_icustays, counts, 'step4_exclude_icu_stays_with_no_events_within_48_hours','step3_exclude_icustays_less_than_48_hours')
counts

In [ ]:
def get_cohort_start_date(icustays):
    df = icustays.groupby(['PERSON_ID']).agg({'VISIT_START_DATETIME': 'min'}).reset_index()
    df.columns = ['SUBJECT_ID','COHORT_START_DATE']
    return df


In [ ]:
def get_cohort_end_date(icustays):
    df = icustays.groupby(['PERSON_ID']).agg({'VISIT_END_DATETIME': 'max'}).reset_index()
    df.columns = ['SUBJECT_ID','COHORT_END_DATE']
    return df


In [ ]:
target_cohort = filtered_icustays.copy()
target_cohort['COHORT_DEFINITION_ID'] = 10001
target_cohort = target_cohort[['COHORT_DEFINITION_ID','PERSON_ID']].drop_duplicates()
target_cohort.columns = ['COHORT_DEFINITION_ID','SUBJECT_ID']

start_date = get_cohort_start_date(icustays)
end_date = get_cohort_end_date(icustays)

target_cohort = target_cohort.merge(start_date, on ='SUBJECT_ID').merge(end_date, on = 'SUBJECT_ID')
target_cohort 

#### Define the outcome cohort

In [ ]:
def get_mortalities(cohort_id):
    query = """
                SELECT {} AS COHORT_DEFINITION_ID,
                       D.PERSON_ID AS SUBJECT_ID,
                       D.DEATH_DATETIME AS COHORT_START_DATE,
                       V.VISIT_END_DATETIME AS COHORT_END_DATE
                FROM (SELECT PERSON_ID, 
                             DEATH_DATETIME
                             FROM omop2.death) D
                INNER JOIN (SELECT  PERSON_ID,
                                    VISIT_END_DATETIME
                            FROM omop2.visit_detail 
                            WHERE visit_detail_concept_id = 32037                          
                                  AND visit_type_concept_id = 2000000006                             
                                  AND discharge_to_concept_id = 4216643) V
                            ON D.PERSON_ID = V.PERSON_ID
            """.format(cohort_id)
    return fetch_data(query)

outcome_cohort = get_mortalities(10002)
outcome_cohort

#### Submit cohorts to DB

In [ ]:
def insert_data(data, connection_details):
    
    connect = database_connector.connect
    conn = connect(connection_details)

    r("""
          insertCohortData <-function(data, conn){

              data$COHORT_START_DATE <- as.Date(data$COHORT_START_DATE, origin = "1970-01-01")
              data$COHORT_END_DATE <- as.Date(data$COHORT_END_DATE, origin = "1970-01-01")
              
              cohort_ids <- unique(data$COHORT_DEFINITION_ID)
              statement <- sprintf("DELETE FROM results.cohort WHERE COHORT_DEFINITION_ID in (%s)", paste(cohort_ids,collapse=", "))
              dbSendStatement(conn,statement)
              
              print(head(data))
              insertTable(conn = conn, 
                databaseSchema = 'results', 
                tableName = 'cohort', 
                data = data,
                dropTableIfExists = FALSE,
                createTable = FALSE,
                tempTable = FALSE
                )
              
          }
      """)
    
    insert_cohort_data = r['insertCohortData']
    insert_cohort_data(data, conn)
    
    return 



In [ ]:
insert_data(target_cohort, connection_details)

In [ ]:
insert_data(outcome_cohort, connection_details)

In [ ]:
query = """
            SELECT *
            FROM results.cohort
            WHERE COHORT_DEFINITION_ID IN (10001,10002)

        """
cohorts = fetch_data(query)
cohorts

In [ ]:
def get_intersect(idx1, idx2):
    idx1_set = set(cohorts['SUBJECT_ID'][cohorts['COHORT_DEFINITION_ID']==idx1])
    idx2_set = set(cohorts['SUBJECT_ID'][cohorts['COHORT_DEFINITION_ID']==idx2])
    return idx1_set.intersection(idx2_set)

In [ ]:
len(get_intersect(10001, 10002))